# 🦈 SHARK CRYPTO ANALYZER - ALL-IN-ONE (2026)
Hệ thống tự động phát hiện và lọc tin tức thị trường Crypto.

**Hướng dẫn:** Nhấn `Runtime` -> `Run all` để chạy toàn bộ.

In [ ]:
# @title 1. Cài đặt thư viện và Cấu hình
!pip install google-genai feedparser playwright pydantic PyYAML aiohttp
!playwright install chromium
!playwright install-deps

import json
import os
import ssl
import asyncio
import time
import yaml
import feedparser
import re
import aiohttp
from datetime import datetime
from google import genai
from google.genai import types, errors
from pydantic import BaseModel
from playwright.async_api import async_playwright

# Mã màu ANSI
BLUE = "\033[94m"
CYAN = "\033[96m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
BOLD = "\033[1m"
RESET = "\033[0m"

# --- CẤU HÌNH TẠI ĐÂY ---
GOOGLE_API_KEY = "NHAP_KEY_O_DAY" # @param {type:"string"}
SHARK_CONTEXT = "BẠN LÀ MÁY LỌC TIN CRYPTO THUẦN TÚY. Chỉ chấp nhận các tin tức trực tiếp về: Bitcoin, Ethereum, Altcoins, Blockchain, DeFi, NFT, sàn giao dịch tiền số. LOẠI BỎ TẤT CẢ các tin về: Chứng khoán truyền thống, Giá Vàng, Ngân hàng trung ương, Bất động sản, hoặc Kinh tế vĩ mô chung TRỪ KHI tin đó nói trực tiếp về ảnh hưởng tới Crypto. Nếu không chắc chắn, hãy loại bỏ. Mục tiêu hàng đầu là thu thập các tin tức quan trọng về thị trường CRYPTO." # @param {type:"string"}
LIMIT_DAYS = 2 # @param {type:"slider", min:1, max:30, step:1}

# @markdown Nhập các link RSS cách nhau bởi dấu phẩy hoặc xuống dòng:
RSS_LINKS_RAW = "https://cointelegraph.com/rss, https://www.coindesk.com/arc/outboundfeeds/rss/, https://cryptoslate.com/feed/, https://vn.beincrypto.com/feed/, https://blog.bitmex.com/feed/" # @param {type:"string"}

# Chuyển chuỗi text thành list
RSS_LINKS = [link.strip() for link in RSS_LINKS_RAW.replace('\n', ',').split(',') if link.strip()]

In [ ]:
# @title 2. Định nghĩa Hệ thống phân tích

class TinPhanTich(BaseModel):
    index: int
    phu_hop: bool

def goi_gemini(noi_dung_prompt, api_key):
    # Cơ chế Retry khi gặp lỗi giới hạn (429)
    max_retries = 3
    for attempt in range(max_retries):
        try:
            client = genai.Client(api_key=api_key)
            config = types.GenerateContentConfig(
                response_mime_type="application/json",
                response_schema=list[TinPhanTich],
            )
            response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=noi_dung_prompt,
                config=config
            )
            return json.loads(response.text) if response.text else None
            
        except Exception as e:
            error_msg = str(e)
            if "429" in error_msg or "RESOURCE_EXHAUSTED" in error_msg:
                wait_time = 60 
                print(f"{YELLOW}⚠️ Gặp lỗi Quota (429). Đang nghỉ giải lao {wait_time} giây trước khi thử lại...{RESET}")
                time.sleep(wait_time)
            else:
                print(f"{RED}!!! Lỗi Gemini: {e}{RESET}")
                if attempt == max_retries - 1: return None
                time.sleep(5)
    return None

async def quet_rss_async(url, session):
    try:
        async with session.get(url, timeout=15, ssl=False) as response:
            html = await response.text()
            return feedparser.parse(html)
    except Exception as e:
        error_msg = str(e)
        if "Header value is too long" in error_msg:
            print(f"{YELLOW}--- Nguồn {url} phản hồi header quá dài (Bỏ qua RSS này/Yahoo)...{RESET}")
        return None

async def lay_noi_dung_chi_tiet(url, context):
    try:
        page = await context.new_page()
        await page.set_extra_http_headers({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/119.0.0.0 Safari/537.36"
        })
        await page.goto(url, wait_until="load", timeout=20000)
        content = await page.evaluate("() => document.body.innerText")
        await page.close()
        return content.strip()
    except:
        return "Không thể bóc tách nội dung."

In [ ]:
# @title 3. Thực thi quy trình lọc tin

async def main():
    if GOOGLE_API_KEY == "NHAP_KEY_O_DAY":
        print(f"{RED}❌ Vui lòng nhập GOOGLE_API_KEY ở Bước 1 trước khi chạy!{RESET}")
        return
        
    print(f"{CYAN}{BOLD}🚀 Đang quét nhanh {len(RSS_LINKS)} nguồn RSS (Đa luồng)...{RESET}")
    kho_tin_tho = []
    tieu_de_da_lay = set()
    
    limit_sec = LIMIT_DAYS * 24 * 3600
    now = time.time()

    connector = aiohttp.TCPConnector(ssl=False)
    async with aiohttp.ClientSession(
        connector=connector, 
        headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/119.0.0.0 Safari/537.36"}
    ) as session:
        tasks = [quet_rss_async(url, session) for url in RSS_LINKS]
        feeds = await asyncio.gather(*tasks)
        
        for feed in feeds:
            if not feed: continue
            for entry in feed.entries:
                tieu_de = entry.get('title', '').strip()
                if not tieu_de or tieu_de in tieu_de_da_lay: continue
                
                published = entry.get('published_parsed')
                if published and (now - time.mktime(published) > limit_sec): continue
                
                kho_tin_tho.append({
                    'tieu_de': tieu_de,
                    'mo_ta': entry.get('summary', '').strip(),
                    'duong_dan': entry.get('link', '')
                })
                tieu_de_da_lay.add(tieu_de)
    
    tong_so_tin = len(kho_tin_tho)
    print(f"{GREEN}✅ Đã lấy và lọc trùng: {tong_so_tin} tin Crypto mới.{RESET}")
    
    if not kho_tin_tho: return

    # LƯU FILE TẤT CẢ TIN ĐÃ TẢI (Lưu trong thư mục, không tải về)
    file_tat_ca = f"tat_ca_tin_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"
    with open(file_tat_ca, 'w', encoding='utf-8') as f:
        json.dump(kho_tin_tho, f, ensure_ascii=False, indent=2)
    print(f"{GREEN}📝 Đã lưu danh sách tất cả tin vào thư mục: {RESET}{BOLD}{file_tat_ca}{RESET}")

    # --- LỌC AI ---
    BATCH_SIZE = 150
    tin_phu_hop = []
    print(f"\n{CYAN}{BOLD}🔄 AI (Gemini) đang lọc nội dung...{RESET}")
    
    for start_idx in range(0, tong_so_tin, BATCH_SIZE):
        end_idx = min(start_idx + BATCH_SIZE, tong_so_tin)
        batch_tin = kho_tin_tho[start_idx:end_idx]
        print(f"  > Check đợt {start_idx//BATCH_SIZE + 1} ({len(batch_tin)} tin)...")
        
        prompt = f"Ngữ cảnh phân tích: {SHARK_CONTEXT}\n\n"
        prompt += f"Hãy lọc ra các tin tức THỰC SỰ liên quan đến thị trường CRYPTO trong danh sách này:\n"
        for i, t in enumerate(batch_tin):
            prompt += f"- Bài {i}: {t['tieu_de']}\n"

        results_ai = goi_gemini(prompt, GOOGLE_API_KEY)
        if results_ai:
            for r in results_ai:
                if r.get('phu_hop'):
                    local_idx = r.get('index')
                    if local_idx is not None and local_idx < len(batch_tin):
                        tin_phu_hop.append(batch_tin[local_idx])
        
        if end_idx < tong_so_tin:
            # Nghỉ cố định 2s cho Key Free
            time.sleep(2)

    # LƯU FILE TIN PHÙ HỢP (Lưu trong thư mục, không tải về)
    if tin_phu_hop:
        file_loc = f"tin_da_loc_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"
        with open(file_loc, 'w', encoding='utf-8') as f:
            json.dump(tin_phu_hop, f, ensure_ascii=False, indent=2)
        print(f"{GREEN}📝 Đã lưu danh sách tin AI lọc vào thư mục: {RESET}{BOLD}{file_loc}{RESET}")

    if tin_phu_hop:
        print(f"\n{CYAN}{BOLD}🎯 Bắt đầu bóc tách {len(tin_phu_hop)} tin phù hợp (Đơn luồng)...{RESET}")
        
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            ctx = await browser.new_context()
            
            for i, t in enumerate(tin_phu_hop):
                print(f"  {YELLOW}[{i+1}/{len(tin_phu_hop)}] --- Đang bóc:{RESET} {t['tieu_de'][:60]}...")
                t['noi_dung'] = await lay_noi_dung_chi_tiet(t['duong_dan'], ctx)
                
            await browser.close()
            
        # FILE KẾT QUẢ CUỐI CÙNG: Lưu và Tải về
        filename = f"bao_cao_crypto_{datetime.now().strftime('%Y-%m-%d')}.txt"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(tin_phu_hop, f, ensure_ascii=False, indent=2)
            
        print(f"\n{GREEN}{BOLD}🎉 HOÀN TẤT! Đã lưu báo cáo tại: {filename}{RESET}")
        print(f"{CYAN}📩 Đang chuẩn bị tải file kết quả cuối cùng về máy...{RESET}")
        try:
            from google.colab import files
            files.download(filename)
        except:
            pass
    else:
        print(f"\n{RED}❌ Không tìm thấy tin nào phù hợp.{RESET}")

await main()